# 데이터 준비
## 데이터 셋 필요성
- 대형 언어 모델은 자연어를 처리하고 이해하는것을 목적으로 설계
- 방대한 양의 텍스트 데이터로 훈련되어, 다양한 질문과 프롬프트에 대한 일관된 응답을 생성하며 맥락에 부합
- 패턴을 분석하여 의미를 추출하며, 인간과 유사한 텍스트를 생성
- 언어번역, 전체 텍스트 생성 및 요약, 감정분석, 질문-답변 시스템에 활용

## 데이터 셋 준비 과정
- 데이터 수집 : 데이터 요구 사항을 신중하게 정의하고 신뢰할 수 있는 웹사이트, 서적, 논문에서 데이터를 수집하여 법적 준수와 개인 정보를 보장
    - 데이터 요구사항 정의
    - 데이터 소스 선택
    - 수집 전략 수립
    - 크롤링 및 수집
    - 데이터 추출과 파싱
    - 인코딩
    - 언어 추출
    - 데이터 백업
    - 개인정보 및 법적 준수
- 데이터 필터링 : 모델 기반 휴리스틱 방법을 사용하여 유해한 자료를 포함하여 품질이 낮거나 관련 없는 컨텐츠를 걸러내고 전반적인 데이터 품질 향상
    - 모델 기반 접근 
    - 휴리스틱 기반 접근
    - 문서 레벨 접근
    - 문장 레벨 접근
- 데이터 중복 제거 : TF-IDF, MinHash, SimHash와 같은 기술을 이용하여 중복되거나 매우 유사한 텍스트를 제거하여 데이터 세트의 고유성 보장
    - TF-IDF 소프트 디커플링
- 데이터 표준화 : 문장 분할, 인코딩 교정, 언어 감지, 불용어 제거 및 철자 교정을 포함하여 텍스트를 통일된 형식으로 변환
- 데이터 검토 : 전처리 단계를 문서화하고 수동 검토를 수행하여 데이터가 품질 표준을 충족하는지 확인하고 추가 개선을 위한 피드백 제공


# 데이터 셋 생성
## 한글 데이터 셋 생성 방법
- 한국 LLM 생태계는 고품질의 fine-tuning 데이터셋 부족
- GPT나 Gemin를 이용하여 데이터 생성 후 활용 가능하지만, 현재 라이선스에서는 위반으로 판단됨
- AI-Hub의 말뭉치를 이용한 데이터 셋 생성 방법
    - 일반 상식 문장 생성 데이터
    - 도서자료 요약
    - 논문자료 요약
    - 문서 요약 텍스트

ai-hub : https://aihub.or.kr/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install datasets


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pandas as pd
import json
from tqdm import tqdm
from datasets import dataset_dict, load_dataset, Dataset, DatasetDict
import random
import threading
import re

/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# default_path= '/content/drive/MyDrive/aihub/'
current_dir = os.getcwd()
default_path = current_dir + "/ai_hub/"
data = os.listdir(default_path)
print(data)

['018.논문자료 요약 데이터']


# 문서요약 텍스트 데이터 셋 생성
- https://aihub.or.kr/aihubdata/data/dwld.do?currMenu=115&topMenu=100

In [18]:
default = default_path + '/문서요약 텍스트/Training'
default
data_foler = os.listdir(default)

In [23]:
default = default_path + '/문서요약 텍스트/Training'
data_folder = os.listdir(default)
df = pd.DataFrame(columns=['input', 'instruction', 'output'])

text_list = []
instruction_list = []
for d in data_folder:
    if d != '.DS_Store' :
      path = default + '/' + d
      sub_df = pd.DataFrame(columns=['input', 'instruction', 'output'])
      print(path)

      with open(path, 'r', encoding='UTF8') as file:
        json_file = json.load(file)

      documents = json_file['documents']

      for i in tqdm(range(len(documents))):
        title = documents[i]['title']
        text = documents[i]['abstractive'][0]
        sentence_list = documents[i]['text']

        number = random.choice([0,1])

        # 난수가 발생하지 않은 경우 내용 요약만 진행
        if number == 0:
          instruction_text ='주어진 문장을 적절하게 요약해주세요.\n\n문장: '
          for sentence in sentence_list:
            try:
              if len(sentence) > 1:
                for sub in sentence:
                  sen = sub['sentence']
                  instruction_text += sen + ' '
              else:
                sen = sentence[0]['sentence']
                instruction_text += sen + ' '
            except:
              pass
          instruction_list.append(instruction_text.strip())
          text_list.append(documents[i]['abstractive'][0])

        # 난수가 발생한 경우 제목 및 내용 요약 진행  
        else:
          instruction_text = '주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: '
          for sentenct in sentence_list:
            try:
              if len(sentenct) > 1:
                for sub in sentenct:
                  sen = sub['sentence']
                  instruction_text += sen + ' '
              else:
                sen = sentenct[0]['sentence']
                instruction_text += sen + ' '
            except:
              pass
            instruction_list.append(instruction_text.strip())
            answer = '제목: ' + title + '\n' + documents[i]['abstractive'][0]
            text_list.append(answer)

      sub_df['input'] = ''
      sub_df['instruction'] = instruction_list
      sub_df['output'] = text_list
      print(sub_df.iloc[0, 1])
      print(sub_df.iloc[0, 2])
      df = pd.concat([df, sub_df], axis=0)
      text_list = []
      instruction_list = []

/Users/wonik/Library/Mobile Documents/iCloud~md~obsidian/Documents/til/TIL/AI/slm_practice/ollama_translator_exam/ai_hub//문서요약 텍스트/Training/train_original 3.json


100%|██████████| 243983/243983 [00:02<00:00, 98162.22it/s] 


주어진 문장을 적절하게 요약해주세요.

문장: ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.
전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주

100%|██████████| 56760/56760 [00:00<00:00, 81124.89it/s]


주어진 문장을 적절하게 요약해주세요.

문장: 이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다. 언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 '공생발전'이란 화두를 연결하면 금방 짐작이 간다. 다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다. 이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다. 최근 미국 프랑스 벨기에 등에서 부유세가 거론되고 독일조차 2년간 한시적으로 5%의 자산세를 거둬 약 155조원을 마련하자는 논의가 있었다. 이런 흐름에 한국만 동떨어져 있기는 어려운 게 글로벌 시대의 특징이다. 항간에는 이번 회동 후 삼성을 비롯해 몇몇 그룹이 노블레스 오블리주 방안을 준비하고 있다는 말이 나도는데 대통령의 강요나 포퓰리즘에 의한 압박보다 자발적 문화로 만들어가야 효과가 큰 법이다. 그런 면에서 재계에 적절한 방안 마련을 맡기고 정치권이나 여론은 너무 압박하지 말고 시간을 줘야 한다. 국가채무 문제로 글로벌 경기 침체 우려가 큰 상황에서 기업들은 '생존'에 큰 부담을 느끼고 있기 때문이다. 이날 전경련에 따르면 30대 그룹은 올해 고용 12만4000명, 투자 114조원 등 '선물'을 준비했다. 세계적인 더블딥이 우려되는 상황에서 공격경영이 어렵겠지만 연초 한 번 발표한 내용을 약간 수정해 내놓은 전경련의 행태는 답답하다. 설립 50주년이 됐으면 좀 더 창의적이고 유연하게 바뀔 때도 됐다. 허창수 전경련 회장은 "대기업ㆍ중소기업이 서로 공생하고 발전할 수 있도록 노력하겠다. 기업이 사회적 책임을 다하겠다"는 원론적인 발언에 그쳐 전경련 특유의 무미건조함을 드러냈다. 한편 이건희 삼성전자 회장

100%|██████████| 24329/24329 [00:00<00:00, 163237.03it/s]


주어진 문장을 적절하게 요약해주세요.

문장: 원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.
원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.


In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
df['input'] = ''

In [26]:
df

,input,instruction,output
0,,주어진 문장을 적절하게 요약해주세요.\n\n문장: ha당 조사료 400만원…작물별 ...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 8억 투입...","제목: 여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성\n여수시는 컬러빌리지 ..."
2,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 8억 투입...","제목: 여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성\n여수시는 컬러빌리지 ..."
3,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 8억 투입...","제목: 여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성\n여수시는 컬러빌리지 ..."
4,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 8억 투입...","제목: 여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성\n여수시는 컬러빌리지 ..."
...,...,...,...
1407150,,주어진 문장을 적절하게 요약해주세요.\n\n문장: 농지개혁법에 의한 농지의 분배불하...,농지개혁법에 의한 농지의 분배불하에 있어서 타인에게 명의를 신탁하여 그 분배를 받는...
1407151,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 1. 쌍무...",제목: 토지소유권이전등기청구사건\n농지개혁법 제19조 제2항 및 동법 시행규칙 제5...
1407152,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 1. 쌍무...",제목: 토지소유권이전등기청구사건\n농지개혁법 제19조 제2항 및 동법 시행규칙 제5...
1407153,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 상속인 또...",제목: 토지건물소유권보존등기등청구사건\n'타인이 상속인이라고 참칭하고 불법하게 상속...


In [27]:
# 데이터 재사용을 위해 parquet 형식으로 저장해서 관리함
df.to_parquet(default_path + '/Save/summary_com.parquet', engine='pyarrow', index=False)

# 논문자료 요약 데이터 corpus 생성
- 기존 AI 허브 데이터셋에서 데이터를 추출해서 corpus 단위로 뭉치는 작업


In [32]:
import os
import pandas as pd
import json
from tqdm import tqdm

default = default_path + '018.논문자료 요약 데이터/01.데이터/1. Training/1. 라벨링데이터_231101_add' 
data_folder = os.listdir(default)

In [34]:
df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])

instruction_list = []
text_list = []
doc_id_list = []
doc_id = 0

for d in data_folder:
    if d != '.DS_Store':
        path = default + '/' + d
        folder = os.listdir(path)

        for f in folder:
            print(f)
            path = default + '/' + d + '/' + f

            sub_df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])


            with open(path, 'r', encoding='UTF8') as file:
                json_file = json.load(file)

            try:
                documents = json_file[0]['data']
            except:
                    documents = json_file['data']

            for i in tqdm(range(len(documents))):
                text1 = documents[i]['summary_section'][0]['orginal_text']
                text_list.append(text1)
                doc_id_list.append('AIHUB_논문자료_' +str(doc_id))
                doc_id += 1

            sub_df['domain'] = ''
            sub_df['doc_id'] = doc_id_list
            sub_df['text'] = text_list
            df = pd.concat([df, sub_df], axis=0)
            text_list = []
            instruction_list = []
            doc_id_list = []

9-N-0.json


100%|██████████| 38000/38000 [00:00<00:00, 1381666.77it/s]


10-N-0.json


100%|██████████| 38000/38000 [00:00<00:00, 1586979.77it/s]


9-N-1.json


100%|██████████| 38000/38000 [00:00<00:00, 1467647.21it/s]


In [35]:
df

,text,doc_id,domain
0,"미국 암 학회(American Cancer Society)에 따르면, 2004년에는...",AIHUB_논문자료_0,NaN
1,종래의 포지티브형 감광성 폴리이미드 수지 전구체의 제조방법으로는 폴리아믹산 에스테르...,AIHUB_논문자료_1,NaN
2,"해양에서 적조를 유발하는 원인 종은 돌말류 (Bacillariophyceae), 침...",AIHUB_논문자료_2,NaN
3,"이동통신 기술 발달 및 이동통신 단말(특히, 휴대폰)의 폭넓은 보급으로, 이동통신 ...",AIHUB_논문자료_3,NaN
4,코덱(codec)은 스피치 코덱(speech codec)과 오디오 코덱(audio ...,AIHUB_논문자료_4,NaN
...,...,...,...
37995,"전자 종이 디스플레이는 액정 디스플레이, 플라즈마 디스플레이 패널, 유기 전계발광 ...",AIHUB_논문자료_113995,NaN
37996,블록 코폴리머의 유도된 자가-조립 (DSA)는 현재의 광학 리소그래피의 상태를 연장...,AIHUB_논문자료_113996,NaN
37997,음성은 인간이 사용하는 가장 보편적이고 편리한 정보전달의 수단이라 할 수 있다. 음...,AIHUB_논문자료_113997,NaN
37998,"각종의 제조 프로세스에 있어서, 규칙적으로 배치된 복수의 워크의 효율적인 반송이 필...",AIHUB_논문자료_113998,NaN


In [ ]:
# default_path = '/content/drive/MyDrive/gdrive/'
# df.to_parquet(default_path + 'Save/논문자료요약.parquet', engine='pyarrow', index=False)

# 일반 상식 문장 생성 데이터

In [ ]:
import random

default_path = '/content/drive/MyDrive/gdrive/'
default = default_path + '048.일반상식 문장 생성 데이터/01-1.정식개방데이터/Training/02.라벨링데이터'
default
data_folder = os.listdir(default)

df = pd.DataFrame(columns =['text', 'doc_id', 'domain'])

instruction_list = []
text_list = []
doc_id_list = []
doc_id = 0


for f in tqdm(data_folder):
  print(f)
  path = default + '/' + f

  sub_df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])

  try:
    with open(path, 'r', encoding='UTF8') as file:
      json_file = json.load(file)

    for documents in json_file:
      text = documents['sentence']
      genSentences = documents['genSentences']
      genChoice = random.choice(genSentences)
      text_list.append(genChoice['label-scenes'].strip())

      instruction_sentence = '임의의 순서대로 나열된 단어들을 보고 적절한 문장으로 재구성하세요.\n\n임의의 순서로 나열된 단어: ['
      for concept in documents['concepts']:
        instruction_sentence += concept['stem'] + ' '
      instruction_sentence = instruction_sentence.strip()[:-1]
      instruction_sentence += ']'

      instruction_list.append(instruction_sentence)
      doc_id_list.append('AIHUB_일반상식문장생성데이터_' +str(doc_id))
      doc_id += 1
  except Exception as e:
    print(e)
    pass

sub_df['domain'] = ''
sub_df['doc_id'] = doc_id_list
sub_df['text'] = text_list
df = pd.concat([df, sub_df], axis=0)
text_list = []
instruction_list = []
doc_id_list = []

In [ ]:
df.to_parquet(default_path+'Save/일반상식문장생성데이터.parquet', engine = 'pyarrow', index = False)

# 도서 요약

In [ ]:
default_path = '/content/drive/MyDrive/gdrive/'
default = default_path + '도서자료 요약/Training'

data_folder = os.listdir(default)
df = pd.DataFrame(columns=['input', 'instruction', 'output'])

text_list = []
instruction_list = []
for d in data_folder:
    path = default + '/' + d
    folder2 = os.listdir(path)

    for f2 in folder2:
        path = default + '/' + d + '/' + f2
        folder3 = os.listdir(path)

        sub_df = pd.DataFrame(columns=['input', 'instruction', 'output'])

        for f3 in tqdm(folder3):
            path = default + '/' + d + '/' + f2 + '/' + f3

            # json 불러오기
            with open(path, 'r', encoding='UTF8') as file:
                json_file = json.load(file)

            number = random.choice([0,1])
            documents = json_file['passage']
            documents_split = documents.split('.')
            documents_split = [d0 for d0 in documents_split if d0 != '']
            if len(documents_split) == 1:
                number = 1
            if '%' in documents:
                number = 1
            if number == 0:
                documents = json_file['passage']
                documents_split = documents.split('.')
                documents_split = [d0 for d0 in documents_split if d0 != '']
                doc_text = ''
                for d0 in documents_split[:-1]:
                    doc_text += d0 + '. '
                text = json_file['summary']
                text_list.append(documents_split[-1].strip())

                instruction_text = '주어진 문장 뒤에 자연스럽게 이어질 문장을 생성해주세요.\n\n문장: '
                instruction_text += doc_text.strip()

                instruction_list.append(instruction_text.strip())

            elif number == 1:
                documents = json_file['passage']
                text = json_file['summary']
                text_list.append(documents.strip())
                title = json_file['metadata']['doc_name']
                label = json_file['metadata']['kdc_label']

                instruction_text = '주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세요.\n\n'
                instruction_text += '제목: ' + title + '\n'
                instruction_text += '요약문: ' + text.strip()

                instruction_list.append(instruction_text.strip())

            else:
                documents = json_file['passage']
                text = json_file['summary']
                text_list.append(text.strip())
                title = json_file['metadata']['doc_name']
                label = json_file['metadata']['kdc_label']

                instruction_text = '주어진 제목과 카테고리에 대한 정보를 토대로, 적합한 문장을 생성해주세요.\n\n'
                instruction_text += '제목: ' + title + '\n'
                instruction_text += '카테고리: ' + '[' + label + ']'

                instruction_list.append(instruction_text.strip())

        sub_df['input'] = ''
        sub_df['instruction'] = instruction_list
        sub_df['output'] = text_list

        print(len(sub_df))

        df = pd.concat([df, sub_df], axis=0)
        text_list = []
        instruction_list = []

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df['input'] = ''

In [ ]:
df

In [ ]:
df.to_parquet(default_path+'Save/summary_com4.parquet', engine = 'pyarrow', index = False)